In [1]:
import os
os.chdir("../..")

In [2]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent
from shell.fleet.data.data_utilize import *
from shell.fleet.data.recv import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
seed_everything(0)

In [4]:
use_contrastive = True
num_tasks = 6



num_init_tasks = 4
dataset_name = "mnist"
num_epochs = 100

data_cfg = {
    "dataset_name": dataset_name,
    "num_tasks": num_tasks,
    "num_train_per_task": 128,
    "num_val_per_task": 102,
    'remap_labels': True,
    'use_contrastive': use_contrastive,
}
dataset = get_dataset(**data_cfg)

INFO:root:Class sequence: [5 0 3 4 2 6 7 1 8 9]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28)


Shared module vs random
1. Shared module can either be trained on [8, 9]
2. Or trained on task 1-4.  

INITIALIZATION
[1,2,3,4] A - B: 

NEW TASK
[5] : local task_5(A) has somethin in common with task_6(B).
[6]: A share B.

In [5]:
net_cfg = {
    'depth': 4,
    'layer_size': 64,
    'num_init_tasks': num_init_tasks,
    'i_size': 28,
    'num_classes': 2,
    'num_tasks': num_tasks,
    'dropout': 0.0,
}

agent_cfg = {
    'memory_size': 64,
    'use_contrastive': use_contrastive,
    'save_dir': 'test',
    "dataset_name": dataset_name,
}

In [6]:
NetCls = MLPSoftLLDynamic
LearnerCls = CompositionalDynamicER
sharing_cfg = DictConfig({
    "scorer": "cross_entropy",
    "num_queries": 4,
    "query_score_threshold": 0.0,
})
train_cfg = {
    # "num_epochs": 40,
    "num_epochs": num_epochs,
    "component_update_freq": num_epochs,
    "num_candidate_modules": 2,
}

agent = Agent(0, 0, dataset,
                NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg, 
                sharing_cfg)

INFO:root:Agent: node_id: 0, seed: 0


In [7]:
## NOTE: there's a bug
## INFO:root:	task: 4	loss: 0.693	acc: 0.491
## for `num_candidate_modules` > 1

for t in range(num_tasks):
    agent.train(t)

INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.69224913	acc: 0.524
INFO:root:	task: avg	loss: 0.69224913	acc: 0.524
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.69224913	acc: 0.524
INFO:root:	task: avg	loss: 0.69224913	acc: 0.524
INFO:root:final components: 4
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 0.69224913	acc: 0.524
INFO:root:	task: 1	loss: 0.69401731	acc: 0.493
INFO:root:	task: avg	loss: 0.69313322	acc: 0.508
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 0.69224913	acc: 0.524
INFO:root:	task: 1	loss: 0.69401731	acc: 0.493
INFO:root:	task: avg	loss: 0.69313322	acc: 0.508
INFO:root:final components: 4
INFO:root:epochs: 0, training task: 2
INFO:root:	task: 0	loss: 0.69224913	acc: 0.524
INFO:root:	task: 1	loss: 0.69401731	acc: 0.493
INFO:root:	task: 2	loss: 0.69849413	acc: 0.481
INFO:root:	task: avg	loss: 0.69492019	acc: 0.499
INFO:root:epochs: 0, training task: 2
INFO:root:	task: 0	loss: 0.69224913	acc: 0.5

NUM_CANDIDATE_MODULES 2


INFO:root:epochs: 1, training task: 4
INFO:root:	task: 0	loss: 0.38669134	acc: 0.971
INFO:root:	task: 1	loss: 0.42410549	acc: 0.987
INFO:root:	task: 2	loss: 0.38842463	acc: 0.972
INFO:root:	task: 3	loss: 0.33672599	acc: 0.985
INFO:root:	task: 4	loss: 0.69322095	acc: 0.491
INFO:root:	task: avg	loss: 0.44583368	acc: 0.881
INFO:root:epochs: 2, training task: 4
INFO:root:	task: 0	loss: 0.38669134	acc: 0.971
INFO:root:	task: 1	loss: 0.42410549	acc: 0.987
INFO:root:	task: 2	loss: 0.38842463	acc: 0.972
INFO:root:	task: 3	loss: 0.33672599	acc: 0.985
INFO:root:	task: 4	loss: 0.69338548	acc: 0.491
INFO:root:	task: avg	loss: 0.44586659	acc: 0.881
INFO:root:epochs: 3, training task: 4
INFO:root:	task: 0	loss: 0.38669134	acc: 0.971
INFO:root:	task: 1	loss: 0.42410549	acc: 0.987
INFO:root:	task: 2	loss: 0.38842463	acc: 0.972
INFO:root:	task: 3	loss: 0.33672599	acc: 0.985
INFO:root:	task: 4	loss: 0.69357982	acc: 0.491
INFO:root:	task: avg	loss: 0.44590545	acc: 0.881
INFO:root:epochs: 4, training task

In [8]:
agent.net.add_tmp_modules(4, 1)

In [9]:
agent.net.structure

ParameterList(
    (0): Parameter containing: [torch.float32 of size 6x4 (GPU 0)]
    (1): Parameter containing: [torch.float32 of size 6x4 (GPU 0)]
    (2): Parameter containing: [torch.float32 of size 6x4 (GPU 0)]
    (3): Parameter containing: [torch.float32 of size 6x4 (GPU 0)]
    (4): Parameter containing: [torch.float32 of size 6x4 (GPU 0)]
)

In [10]:
agent.net.components

ModuleList(
  (0-5): 6 x Linear(in_features=64, out_features=64, bias=True)
)

In [11]:
agent.agent.optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0

Parameter Group 2
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [12]:
testloaders = {task: torch.utils.data.DataLoader(testset,
                                                         batch_size=128,
                                                         shuffle=False,
                                                         num_workers=0,
                                                         pin_memory=True,
                                                         ) for task, testset in enumerate(dataset.testset[:num_tasks])}

eval_net(agent.net, testloaders)

{0: 0.9706196581196581,
 1: 0.9849397590361446,
 2: 0.9743718592964824,
 3: 0.986130374479889,
 4: 0.9016641452344932,
 'avg': 0.9635451592333336}

In [13]:

target = torch.tensor([0.0, 1.0])
pred = torch.tensor([0.5, 0.5])

loss = torch.nn.functional.binary_cross_entropy_with_logits(pred, target)
loss

tensor(0.7241)